# This notebook explores results from the 2018 cycling season

Data<sup>1</sup> include categorized races performed by all riders in World Tour teams<sup>2</sup>. This include 18 teams with a total of 522 riders.
#### The goal is to find features that can predict the winner (maybe top 10 is better) of Tour de France (TdF).  
<sup>1</sup>data is scraped from CQranking  
<sup>2</sup>Pro Continental teams are excluded for now

In [147]:
import pandas as pd
import numpy as np

In [148]:
pd.set_option('max_colwidth', 500)
pd.set_option('display.width', 100)
pd.set_option('display.expand_frame_repr', False)
pd.set_option("display.max_rows", 100)

In [149]:
# @hidden_cell

df_riders = pd.read_csv(r'E:\cycling\scraper\riders_2018.csv',encoding='iso-8859-1')
df_rider_results = pd.read_csv(r'E:\cycling\scraper\rider_results_2018.csv',encoding='iso-8859-1')

In [150]:
df_riders

,team,Rider,Date of birth,rank_start,point_start,rank_end,point_end,Comments,Country
0,ALM,BAGDONAS Gediminas,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
1,ALM,BAKELANTS Jan,14/02/1986,117.0,472.0,565.0,120.0,NaN,BEL
2,ALM,BARBIER Rudy,18/12/1992,104.0,506.0,717.0,90.0,NaN,FRA
3,ALM,BARDET Romain,09/11/1990,30.0,1162.0,14.0,1492.0,NaN,FRA
4,ALM,BIDARD François,19/03/1992,781.0,80.0,523.0,137.0,NaN,FRA
5,ALM,BOUCHARD Geoffrey,01/04/1992,NaN,NaN,1140.0,42.0,Trainee as from 31/07,FRA
6,ALM,CHAMPOUSSIN Clément,29/05/1998,2949.0,2.0,978.0,55.0,Trainee as from 31/07,FRA
7,ALM,CHEREL Mikael,17/03/1986,552.0,127.0,492.0,147.0,NaN,FRA
8,ALM,CHEVRIER Clément,29/06/1992,844.0,70.0,1416.0,25.0,NaN,FRA
9,ALM,COSNEFROY Benoit,17/10/1995,375.0,191.0,311.0,229.0,NaN,FRA


In [151]:
df_rider_results.head()

,Date,Race,Rank,Rider,CQ
0,6/10/2018,Tour de Vendée,23.,BAGDONAS Gediminas,0
1,4/10/2018,Paris - Bourges,70.,BAGDONAS Gediminas,0
2,26/09/2018,World Championships (Innsbruck) I.T.T.,51.,BAGDONAS Gediminas,5
3,23/09/2018,World Championships (Innsbruck) T.T.T.,15.,BAGDONAS Gediminas,7
4,12/09/2018,GP de Wallonie,107.,BAGDONAS Gediminas,0


In [152]:
df_results = df_rider_results.merge(df_riders,on='Rider',how='inner',suffixes=('_l','_r'))

In [153]:
df_results

,Date,Race,Rank,Rider,CQ,team,Date of birth,rank_start,point_start,rank_end,point_end,Comments,Country
0,6/10/2018,Tour de Vendée,23.,BAGDONAS Gediminas,0,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
1,4/10/2018,Paris - Bourges,70.,BAGDONAS Gediminas,0,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
2,26/09/2018,World Championships (Innsbruck) I.T.T.,51.,BAGDONAS Gediminas,5,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
3,23/09/2018,World Championships (Innsbruck) T.T.T.,15.,BAGDONAS Gediminas,7,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
4,12/09/2018,GP de Wallonie,107.,BAGDONAS Gediminas,0,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
5,1/09/2018,Brussels Cycling Classic,70.,BAGDONAS Gediminas,0,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
6,24/08/2018,Tour du Poitou Charentes,6.,BAGDONAS Gediminas,45,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
7,23/08/2018,"Tour du Poitou Charentes, Stage 4 : Champagné-Saint-Hilaire - Couhé I.T.T.",9.,BAGDONAS Gediminas,0,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
8,23/08/2018,"Tour du Poitou Charentes, Stage 3 : Gençay - Couhé",20.,BAGDONAS Gediminas,0,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
9,22/08/2018,"Tour du Poitou Charentes, Stage 2 : Segonzac - Melle",15.,BAGDONAS Gediminas,0,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU


Clean up the data types

Replace below values  
leader (leader of the stage race): 0  
\- (usually means TTT): nan  
DNF (did not finish): -1  
OOT (out of time): -2  
DNS (did not start): -5  
DQ (disqualified): -10

In [154]:
df_results['Rank'] = df_results['Rank'].replace('leader',0)
df_results['Rank'] = df_results['Rank'].replace('DNF',-1)
df_results['Rank'] = df_results['Rank'].replace('OOT',-2)
df_results['Rank'] = df_results['Rank'].replace('-',np.nan)
df_results['Rank'] = df_results['Rank'].replace('DNS',-5)
df_results['Rank'] = df_results['Rank'].replace('DQ',-10)
df_results['Rank'] = df_results['Rank'].astype(float)

In [155]:
df_results['Date'] = pd.to_datetime(df_results['Date'],format='%d/%m/%Y')

TdF results

In [156]:
df_tdf_gc = df_results.loc[df_results['Race']=='Tour de France',['Rider','Rank']]

In [157]:
df_tdf_gc.sort_values('Rank')

,Rider,Rank
25245,THOMAS Geraint,1.0
25720,DUMOULIN Tom,2.0
24110,FROOME Chris,3.0
23036,ROGLIC Primoz,4.0
22587,KRUIJSWIJK Steven,5.0
128,BARDET Romain,6.0
14372,LANDA MEANA Mikel,7.0
29786,MARTIN Daniel,8.0
21861,ZAKARIN Ilnur,9.0
14649,QUINTANA ROJAS Nairo Alexander,10.0


Riders who participated in TdF

In [158]:
tdf_riders = df_results.loc[df_results['Race'].str.contains('Tour de France'),'Rider'].unique()

In [159]:
print(len(tdf_riders))
tdf_riders

144


array([u'BARDET Romain', u'DILLIER Silvan', u'DOMONT Axel',
       u'FRANK Mathias', u'GALLOPIN Tony', u'LATOUR Pierre',
       u'NAESEN Oliver', u'VUILLERMOZ Alexis', u'CORT NIELSEN Magnus',
       u'FRAILE MATARRANZ Omar', u'FUGLSANG Jakob', u'GRUZDEV Dmitriy',
       u'HANSEN Jesper', u'KANGERT Tanel', u'SANCHEZ GIL Luis Leon',
       u'VALGREN ANDERSEN Michael', u'COLBRELLI Sonny',
       u'HAUSSLER Heinrich', u'IZAGIRRE INSAUSTI Gorka',
       u'IZAGIRRE INSAUSTI Ion', u'KOREN Kristjan', u'NIBALI Vincenzo',
       u'PELLIZOTTI Franco', u'POZZOVIVO Domenico', u'BEVIN Patrick',
       u'CARUSO Damiano', u'GERRANS Simon', u'K\xdcNG Stefan',
       u'PORTE Richie', u'SCH\xc4R Michael', u'VAN AVERMAET Greg',
       u'VAN GARDEREN Tejay', u'BODNAR Maciej', u'BURGHARDT Marcus',
       u'MAJKA Rafal', u'M\xdcHLBERGER Gregor', u'OSS Daniel',
       u'POLJANSKI Pawel', u'P\xd6STLBERGER Lukas', u'SAGAN Peter',
       u'DEMARE Arnaud', u'GAUDU David', u'GUARNIERI Jacopo',
       u'LE GAC Oliv

Top ten finishers

In [160]:
top10_riders = df_tdf_gc.loc[df_tdf_gc['Rank'].between(1,10),'Rider'].values

In [161]:
top10_riders

array([u'BARDET Romain', u'LANDA MEANA Mikel',
       u'QUINTANA ROJAS Nairo Alexander', u'ZAKARIN Ilnur',
       u'KRUIJSWIJK Steven', u'ROGLIC Primoz', u'FROOME Chris',
       u'THOMAS Geraint', u'DUMOULIN Tom', u'MARTIN Daniel'], dtype=object)

Non top ten finishers

In [162]:
non_top10_riders = list(set(tdf_riders) - set(top10_riders))
non_top10_riders

[u'SCH\xc4R Michael',
 u'KWIATKOWSKI Michal',
 u'VANENDERT Jelle',
 u'GOGL Michael',
 u'BOASSON HAGEN Edvald',
 u'VANMARCKE Sep',
 u'HAYMAN Mathew',
 u'MARTINEZ POVEDA Daniel Felipe',
 u'GESCHKE Simon',
 u'CRADDOCK Lawson',
 u'HANSEN Jesper',
 u'CARUSO Damiano',
 u'ALAPHILIPPE Julian',
 u'VALGREN ANDERSEN Michael',
 u'GROENEWEGEN Dylan',
 u'ARNDT Nikias',
 u'SAGAN Peter',
 u'OSS Daniel',
 u'ROLLAND Pierre',
 u'BODNAR Maciej',
 u'MARTIN Tony',
 u'RICHEZE ARAQUISTAIN Maximiliano Ariel',
 u'FRAILE MATARRANZ Omar',
 u'BENOOT Tiesj',
 u'SINKELDAM Ramon',
 u'KANGERT Tanel',
 u'DILLIER Silvan',
 u'NIEVE ITURRALDE Mikel',
 u'KOREN Kristjan',
 u'FRANK Mathias',
 u'STUYVEN Jasper',
 u'GILBERT Philippe',
 u'DEGENKOLB John',
 u'CORT NIELSEN Magnus',
 u'GREIPEL Andr\xe9',
 u'IZAGIRRE INSAUSTI Gorka',
 u'LE GAC Olivier',
 u'MOLARD Rudy',
 u'MATTHEWS Michael',
 u'CAVENDISH Mark',
 u'URAN URAN Rigoberto',
 u'ROJAS GIL Jose Joaquin',
 u'ROOSEN Timo',
 u'DURASEK Kristijan',
 u'BENNATI Daniele',
 u'TROIA

Find races before TdF (2018-07-07)

In [163]:
mask = (df_results['Date'] < '2018-07-07')
df_pre_tdf = df_results.loc[mask]

In [164]:
df_pre_tdf

,Date,Race,Rank,Rider,CQ,team,Date of birth,rank_start,point_start,rank_end,point_end,Comments,Country
22,2018-06-24,National Championships Lithuania (Gargzdai) R.R.,1.0,BAGDONAS Gediminas,30,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
23,2018-06-22,National Championships Lithuania (Gargzdai) I.T.T.,1.0,BAGDONAS Gediminas,15,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
24,2018-06-03,Boucles de la Mayenne,5.0,BAGDONAS Gediminas,50,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
25,2018-06-03,"Boucles de la Mayenne, Stage 3 : Congrier - Laval",15.0,BAGDONAS Gediminas,0,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
26,2018-06-02,"Boucles de la Mayenne, Stage 2 : Saint-Aignan-de-Couptrain - Pré-en-Pail-Saint-Samson",13.0,BAGDONAS Gediminas,0,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
27,2018-06-01,"Boucles de la Mayenne, Stage 1 : Saint-Berthevin - Gorron",10.0,BAGDONAS Gediminas,0,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
28,2018-05-31,"Boucles de la Mayenne, Prologue : Laval I.T.T.",8.0,BAGDONAS Gediminas,0,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
29,2018-05-26,GP de Plumelec-Morbihan,96.0,BAGDONAS Gediminas,0,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
30,2018-05-13,4 Jours de Dunkerque,39.0,BAGDONAS Gediminas,5,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU
31,2018-05-01,Rund um den Finanzplatz Eschborn - Frankfurt,-1.0,BAGDONAS Gediminas,0,ALM,26/12/1985,510.0,140.0,317.0,226.0,NaN,LTU


Calculate rider performance in pre-TdF races

In [165]:
points_sum = []
race_days = []
wins = []
podiums = []
top_tens = []
for r in tdf_riders:
    df_r = df_pre_tdf[df_pre_tdf['Rider']==r]
    points_sum.append(df_r['CQ'].sum())
    race_days.append(df_r['Race'].count())
    wins.append(df_r.loc[df_r['Rank']==1,'Race'].count())
    podiums.append(df_r.loc[df_r['Rank'].between(1,3),'Race'].count())
    top_tens.append(df_r.loc[df_r['Rank'].between(1,10),'Race'].count())
   
df_performance = pd.DataFrame(
    {'riders': tdf_riders,
     'win': wins,
     'podium': podiums,
     'top_ten': top_tens,
     'points': points_sum,
     'race_days': race_days,
    },columns=['riders','win','podium','top_ten','points','race_days'])

In [166]:
df_performance

,riders,win,podium,top_ten,points,race_days
0,BARDET Romain,1,6,17,772,32
1,DILLIER Silvan,1,3,6,371,38
2,DOMONT Axel,0,0,0,33,41
3,FRANK Mathias,0,0,8,262,52
4,GALLOPIN Tony,2,5,9,325,37
5,LATOUR Pierre,1,3,18,601,43
6,NAESEN Oliver,0,2,7,433,41
7,VUILLERMOZ Alexis,0,3,8,340,38
8,CORT NIELSEN Magnus,2,6,17,424,47
9,FRAILE MATARRANZ Omar,2,4,4,225,37


In [167]:
df_performance.sort_values(by=['win', 'podium','top_ten'],ascending=False)

,riders,win,podium,top_ten,points,race_days
71,VALVERDE BELMONTE Alejandro,11,17,26,1600,42
105,GROENEWEGEN Dylan,9,10,11,438,29
61,IMPEY Daryl,8,16,19,838,49
109,ROGLIC Primoz,7,14,18,982,37
115,KWIATKOWSKI Michal,7,11,17,805,43
74,GAVIRIA RENDON Fernando,7,10,12,312,43
39,SAGAN Peter,6,14,26,1246,43
112,BERNAL GOMEZ Egan Arley,6,11,20,866,39
51,GREIPEL André,6,10,16,438,46
30,VAN AVERMAET Greg,5,10,21,785,47
